In [8]:
import copy
import json
import zstd
import numpy as np

In [2]:
from core.data_utils import Game

In [82]:
class Game2(Game):
    def get_training_frames(self, pid=None, pname=None, v=None):
        # pid OR name. Bot version is option (only when name given)
        # if all None, take random player

        if pid is None and pname is not None:
            for p in self.meta_data['players']:
                if pname == p['name'].split(' ')[0]:
                    pid = p['player_id']
                    break
                    
        assert pid is not None, "Error: player not found in replay"
        
        map_shape = self.production.shape[1], self.production.shape[2]
        factories = np.zeros((*map_shape, 1), dtype=np.float32)
        for fx, fy in self.factories:
            factories[fy, fx] = -1 # Assume all are enemies

        # Then set for the player of interest
        fx, fy = self.factories[pid]
        factories[fy, fx] = 1

        # normalize some of the arrays
        production = (self.production - 500.)/1000. # Guessing on norm values for now
        
        entities = self.entities.copy()
        
        my_ships = entities[:, :, :, 2+pid].copy()
        
        print(np.max(my_ships))
        #print(my_ships[300, 25:50, 25:50])
        print(my_ships[300, 15:60, 15:60])

        has_ship = np.sum(entities[:, :, :, 2:].copy().astype(np.float32), -1)
        
        print(has_ship[300, 15:60, 15:60])
        
        has_ship_mask = has_ship.copy()
        
        # Convert enemy ships
        has_ship = -1 * has_ship
        has_ship[my_ships>0.5] = 1
        
        print(has_ship[300, 15:60, 15:60])
        
        # Normalize
        entity_energies = (entities[:, :, :, 0].copy().astype(np.float32) - 500.)/500.
        entity_energies *= has_ship_mask

        has_ship = np.expand_dims(has_ship, -1)
        entity_energies = np.expand_dims(entity_energies, -1)
        
        print(entity_energies[300, 15:60, 15:60])

        dropoffs = self.dropoffs
        has_dropoff = np.zeros((production.shape[0], *map_shape, 1), dtype=np.float32)
        assert len(dropoffs) == len(production)
        for ix, ds in enumerate(dropoffs):
            for d in ds:
                x, y, oid = d
                v = 1. if oid == pid else -1.
                has_dropoff[ix, y, x] = v

        # factories need to be duplicated across frames
        factories = np.repeat(np.expand_dims(factories, 0), production.shape[0], 0)

        production = np.expand_dims(production, -1)

        frames = np.concatenate([production, has_ship, entity_energies, factories, has_dropoff], axis=-1)


In [83]:
#game = GameTemp()
game = Game2()

In [84]:
path = '/Users/Peace/Desktop/replays/20181019/{}'
#path = path.format('ts2018-halite-3-gold-replays_replay-20181014-000921%2B0000-1539475028-40-40')
#path = path.format('ts2018-halite-3-gold-replays_replay-20181014-002314%2B0000-1539475868-40-40')
path = path.format('ts2018-halite-3-gold-replays_replay-20181019-000050%2B0000-1539907246-32-32')

game.load_replay(path)

In [85]:
game.get_training_frames(pname='mawei3')

1
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [43]:
game.meta_data.keys()

dict_keys(['ENGINE_VERSION', 'GAME_CONSTANTS', 'REPLAY_FILE_VERSION', 'game_statistics', 'map_generator_seed', 'number_of_players', 'players'])

In [18]:
game.replay.keys()

dict_keys(['ENGINE_VERSION', 'GAME_CONSTANTS', 'REPLAY_FILE_VERSION', 'full_frames', 'game_statistics', 'map_generator_seed', 'number_of_players', 'players', 'production_map'])

In [246]:
replay = game.get_replay()
replay['full_frames'][0]['entities']
#frame['entities'][str(pid)]

{}

In [247]:
np.max(game.entities[41, :, :, 0])

553

In [248]:
print(game.deposited[-4:])

[[151756, 131830, 97905, 96404], [152136, 131830, 98617, 97010], [152617, 131844, 98617, 97026], [153580, 131844, 99202, 97306]]


In [44]:
# Frame 6 (0 indexed) corresponds to frame 7 in game (1 indexed)
for i in range(113, 118):
    print('Frame: {}'.format(i))
    print(game.events[i]) # Happens after moves are made
    #print(game.production[i, 21, 12])
    print(game.moves[i, 15, 3:5])
    #print(game.generate[i])
    print(game.entities[i, 15, 3])
    #print(game.energy[i])
    #print(game.deposited[i])
    print(game.dropoffs[i]) # Current for this frame
    #print(game.production_compressed)

Frame: 113
[[0, 42, 42, 68, 3], [0, 21, 42, 69, 2]]
[[0 0 0 0]
 [0 0 0 0]]
[0 0 0 0 0 0]
[]


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [180]:
game.dropoffs[-1]

[[8, 0, 0], [34, 38, 0]]

In [45]:
print(len(game.events))
print(game.production.shape)
print(game.moves.shape)
print(game.generate.shape)
print(len(game.entities))
print(len(game.energy))
print(len(game.deposited))
print(game.factories.shape)
print(len(game.dropoffs))
print(len(game.production_compressed))

501
(500, 64, 64)
(500, 64, 64, 4)
(500, 4)
500
501
501
(4, 2)
500
502


In [29]:
game.initial_production.dtype

dtype('int64')

In [32]:
game.initial_production = game.initial_production.astype('int16')

In [47]:
np.max(game.initial_production)

983

In [52]:
import pickle
import gzip
#game.replay = None
with gzip.open('/Users/Peace/Desktop/test.pkl', 'wb') as outfile:
    pickle.dump(game.entities, outfile)

In [78]:
game.moves[0].shape

(425, 40, 40, 2)

In [12]:
np.max(game.production)

3077

In [18]:
game.meta_data

{'ENGINE_VERSION': '0.6.2.prelaunch.51.g40db6',
 'GAME_CONSTANTS': {'CAPTURE_ENABLED': False,
  'CAPTURE_RADIUS': 3,
  'DEFAULT_MAP_HEIGHT': 40,
  'DEFAULT_MAP_WIDTH': 40,
  'DROPOFF_COST': 4000,
  'DROPOFF_PENALTY_RATIO': 4,
  'EXTRACT_RATIO': 4,
  'FACTOR_EXP_1': 2.0,
  'FACTOR_EXP_2': 2.0,
  'INITIAL_ENERGY': 5000,
  'INSPIRATION_ENABLED': True,
  'INSPIRATION_RADIUS': 4,
  'INSPIRATION_SHIP_COUNT': 2,
  'INSPIRED_BONUS_MULTIPLIER': 2.0,
  'INSPIRED_EXTRACT_RATIO': 4,
  'INSPIRED_MOVE_COST_RATIO': 10,
  'MAX_CELL_PRODUCTION': 1000,
  'MAX_ENERGY': 1000,
  'MAX_PLAYERS': 16,
  'MAX_TURNS': 425,
  'MAX_TURN_THRESHOLD': 64,
  'MIN_CELL_PRODUCTION': 900,
  'MIN_TURNS': 400,
  'MIN_TURN_THRESHOLD': 32,
  'MOVE_COST_RATIO': 10,
  'NEW_ENTITY_ENERGY_COST': 1000,
  'PERSISTENCE': 0.7,
  'SHIPS_ABOVE_FOR_CAPTURE': 3,
  'STRICT_ERRORS': False},
 'REPLAY_FILE_VERSION': 3,
 'game_statistics': {'number_turns': 426,
  'player_statistics': [{'all_collisions': 41,
    'average_entity_distance': 20,

In [4]:
parsed_replay.keys()

dict_keys(['ENGINE_VERSION', 'GAME_CONSTANTS', 'REPLAY_FILE_VERSION', 'full_frames', 'game_statistics', 'map_generator_seed', 'number_of_players', 'players', 'production_map'])

In [5]:
meta_data = {key: parsed_replay[key] for key in ['ENGINE_VERSION', 'GAME_CONSTANTS', 'REPLAY_FILE_VERSION',
                                                'game_statistics', 'map_generator_seed',
                                                'number_of_players', 'players']}

In [23]:
print(meta_data.keys())

dict_keys(['ENGINE_VERSION', 'GAME_CONSTANTS', 'REPLAY_FILE_VERSION', 'game_statistics', 'map_generator_seed', 'number_of_players', 'players'])


In [6]:
parsed_replay['full_frames'][0].keys()

dict_keys(['cells', 'deposited', 'energy', 'entities', 'events', 'moves'])

In [43]:
parsed_replay['full_frames'][2]#['energy']

{'cells': [],
 'deposited': {'0': 0, '1': 0, '2': 0, '3': 0},
 'energy': {'0': 3000, '1': 3000, '2': 4000, '3': 4000},
 'entities': {'0': {'2': {'energy': 0,
    'is_inspired': False,
    'x': 14,
    'y': 14}},
  '1': {'3': {'energy': 0, 'is_inspired': False, 'x': 33, 'y': 14}},
  '2': {'1': {'energy': 0, 'is_inspired': False, 'x': 14, 'y': 33}},
  '3': {'0': {'energy': 0, 'is_inspired': False, 'x': 33, 'y': 33}}},
 'events': [{'energy': 0,
   'id': 4,
   'location': {'x': 33, 'y': 14},
   'owner_id': 1,
   'type': 'spawn'},
  {'energy': 0,
   'id': 5,
   'location': {'x': 14, 'y': 14},
   'owner_id': 0,
   'type': 'spawn'}],
 'moves': {'0': [{'direction': 'n', 'id': 2, 'type': 'm'}, {'type': 'g'}],
  '1': [{'direction': 'n', 'id': 3, 'type': 'm'}, {'type': 'g'}],
  '2': [{'direction': 'e', 'id': 1, 'type': 'm'}],
  '3': [{'direction': 'w', 'id': 0, 'type': 'm'}]}}

In [8]:
parsed_replay['production_map']['grid'][0][0].keys()

dict_keys(['energy'])

In [9]:
assert list(parsed_replay['production_map']['grid'][0][0].keys()) == ['energy'] # Ensure no surprises here

In [20]:
raw_energy_grid = parsed_replay['production_map']['grid']
energy_grid = []
for row in raw_energy_grid:
    energy_grid.append([x['energy'] for x in row])
first_frame = np.array(energy_grid)

frames = []
for frame in parsed_replay['full_frames']:
    prev_frame = first_frame if len(frames) == 0 else frames[-1]
    current_frame = prev_frame.copy()
    for c in frame['cells']:
        current_frame[c['y'], c['x']] = c['production']    
    frames.append(current_frame)


In [33]:
np.array(frames)[203, 25, 28]

85

In [69]:
map_shape = np.array(frames).shape[1:]

In [133]:
def parse_events(frame):
    frame_events = []
    for e in frame['events']:
        assert e['type'] in ['spawn', 'shipwreck'], print(e)
        if 'owner_id' in e: # spawn
            event = [1, e['location']['x'], e['location']['y'], e['id'], e['owner_id']]
        else: # shipwreck
            event = [0, e['location']['x'], e['location']['y']] + [x for x in e['ships']]
        frame_events.append(event)
    return frame_events

In [ ]:
events = []
for frame in parsed_replay['full_frames']: # [:3]
    frame_events = parse_events(frame)
    events.append(frame_events)

In [59]:
deposited = [x['deposited'] for x in parsed_replay['full_frames']]
deposited = [[x[y] if y in x else 0 for y in ['0', '1', '2', '3']] for x in deposited]

In [56]:
np.array(deposited)

array([[     0,      0,      0,      0],
       [     0,      0,      0,      0],
       [     0,      0,      0,      0],
       ...,
       [ 42068, 146814,  69898,   1294],
       [ 42068, 146842,  69898,   1294],
       [ 42068, 146842,  69898,   1294]])

In [97]:
meta_data['number_of_players']

4

In [107]:


def parse_moves(parsed_replay):

    num_players = len(parsed_replay['full_frames'][1]['moves']) #meta_data['number_of_players']

    valid_moves = ['o', 'n', 'e', 's', 'w']
    moves = []
    generate = np.zeros((len(parsed_replay['full_frames']) - 2, num_players), dtype=np.uint8)
    for ix, frame in enumerate(parsed_replay['full_frames'][1:-1]): # No moves on first or last frames
        frame_moves = np.zeros((*map_shape, num_players), dtype=np.uint8)
        for pid in range(num_players):
            if str(pid) not in frame['moves']:
                continue
            for move in frame['moves'][str(pid)]:

                if move['type'] == 'm':
                    mid = move['id']
                    ent = frame['entities'][str(pid)][str(mid)]
                    assert move['direction'] in valid_moves
                    frame_moves[ent['y'], ent['x'], pid] = valid_moves.index(move['direction'])
                else:
                    generate[ix, pid] = 1
        moves.append(frame_moves)

    return np.array(moves), generate

    
    

4


(450, 48, 48, 4)

In [108]:
parsed_replay['full_frames'][1].keys()

dict_keys(['cells', 'deposited', 'energy', 'entities', 'events', 'moves'])

In [122]:
parsed_replay['full_frames'][2]['entities']

{'0': {'2': {'energy': 0, 'is_inspired': False, 'x': 14, 'y': 14}},
 '1': {'3': {'energy': 0, 'is_inspired': False, 'x': 33, 'y': 14}},
 '2': {'1': {'energy': 0, 'is_inspired': False, 'x': 14, 'y': 33}},
 '3': {'0': {'energy': 0, 'is_inspired': False, 'x': 33, 'y': 33}}}

In [123]:
def parse_entities(parsed_replay):

    num_players = len(parsed_replay['full_frames'][1]['moves']) #meta_data['number_of_players']
    num_features = 2
    entities = []
    for ix, frame in enumerate(parsed_replay['full_frames'][1:-1]): # No enties on first frame (last doesn't matter)
        frame_entities = np.zeros((*map_shape, num_features+num_players), dtype=np.uint8)
        for pid in range(num_players):
            for ent in frame['entities'][str(pid)]:
                ent = frame['entities'][str(pid)][ent]
                frame_entities[ent['y'], ent['x'], 0] = ent['energy']
                frame_entities[ent['y'], ent['x'], 1] = int(ent['is_inspired'])
                frame_entities[ent['y'], ent['x'], pid+num_features] = 1
        entities.append(frame_entities)

    return np.array(entities)

In [127]:
parse_entities(parsed_replay)[100, 20]

array([[  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [233,   0,   1,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [167,   0,   1,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   

In [120]:
energy = [[f['energy'][y] for y in sorted(f['energy'].keys())] for f in parsed_replay['full_frames']]
energy = np.array(energy)

In [ ]:
entities = []
for frame in parsed_replay['full_frames']:
    frame[]

In [119]:
np.array(energy)

array([[ 5000,  5000,  5000,  5000],
       [ 4000,  4000,  4000,  4000],
       [ 3000,  3000,  4000,  4000],
       ...,
       [18068, 92814, 54898,   294],
       [18068, 92842, 54898,   294],
       [18068, 92842, 54898,   294]])

In [24]:
parsed_replay['full_frames'][-1]

{'cells': [],
 'deposited': {'0': 42068, '1': 146842, '2': 69898, '3': 1294},
 'energy': {'0': 18068, '1': 92842, '2': 54898, '3': 294},
 'entities': {'0': {'47': {'energy': 0,
    'is_inspired': False,
    'x': 14,
    'y': 14},
   '70': {'energy': 612, 'is_inspired': False, 'x': 13, 'y': 13},
   '81': {'energy': 610, 'is_inspired': False, 'x': 13, 'y': 14},
   '87': {'energy': 633, 'is_inspired': False, 'x': 14, 'y': 15},
   '93': {'energy': 633, 'is_inspired': False, 'x': 15, 'y': 14}},
  '1': {'100': {'energy': 33, 'is_inspired': False, 'x': 30, 'y': 11},
   '103': {'energy': 17, 'is_inspired': False, 'x': 28, 'y': 5},
   '104': {'energy': 6, 'is_inspired': False, 'x': 30, 'y': 4},
   '105': {'energy': 12, 'is_inspired': False, 'x': 33, 'y': 15},
   '112': {'energy': 32, 'is_inspired': False, 'x': 43, 'y': 22},
   '15': {'energy': 12, 'is_inspired': False, 'x': 37, 'y': 8},
   '25': {'energy': 24, 'is_inspired': False, 'x': 32, 'y': 13},
   '52': {'energy': 1, 'is_inspired': False,

In [76]:
def parse_replay(data, player_names):
    
    for player in player_names:

        print("Load Basic Information")
        player = [p for p in data['players'] if p['name'].split(" ")[0] == player_name][0]
        player_id = int(player['player_id'])
        my_shipyard = hlt.Shipyard(player_id, None,
                                   hlt.Position(player['factory_location']['x'], player['factory_location']['y']))
        other_shipyards = [
            hlt.Shipyard(p['player_id'], None, hlt.Position(p['factory_location']['x'], p['factory_location']['y']))
            for p in data['players'] if int(p['player_id']) != player_id]
        width = data['production_map']['width']
        height = data['production_map']['height']



        print("Load Player Ships")
        moves = [{} if str(player_id) not in f['moves'] else {m['id']: m['direction'] for m in f['moves'][str(player_id)] if
                                                              m['type'] == "m"} for f in data['full_frames']]
        ships = [{} if str(player_id) not in f['entities'] else {
            int(sid): hlt.Ship(player_id, int(sid), hlt.Position(ship['x'], ship['y']), ship['energy']) for sid, ship in
            f['entities'][str(player_id)].items()} for f in data['full_frames']]

        print("Load Other Player Ships")
        other_ships = [
            {int(sid): hlt.Ship(int(pid), int(sid), hlt.Position(ship['x'], ship['y']), ship['energy']) for pid, p in
             f['entities'].items() if
             int(pid) != player_id for sid, ship in p.items()} for f in data['full_frames']]

        print("Load Droppoff Information")
        first_my_dropoffs = [my_shipyard]
        first_them_dropoffs = other_shipyards
        my_dropoffs = []
        them_dropoffs = []
        for f in data['full_frames']:
            new_my_dropoffs = copy.deepcopy(first_my_dropoffs if len(my_dropoffs) == 0 else my_dropoffs[-1])
            new_them_dropoffs = copy.deepcopy(first_them_dropoffs if len(them_dropoffs) == 0 else them_dropoffs[-1])
            for e in f['events']:
                if e['type'] == 'construct':
                    if int(e['owner_id']) == player_id:
                        new_my_dropoffs.append(
                            hlt.Dropoff(player_id, None, hlt.Position(e['location']['x'], e['location']['y'])))
                    else:
                        new_them_dropoffs.append(
                            hlt.Dropoff(e['owner_id'], None, hlt.Position(e['location']['x'], e['location']['y'])))
            my_dropoffs.append(new_my_dropoffs)
            them_dropoffs.append(new_them_dropoffs)
        yield list(zip(frames, moves, ships, other_ships, my_dropoffs, them_dropoffs))

In [77]:
player_names = [x['name'].split(' ')[0] for x in meta_data['players']]

In [83]:
frames = extract_frames(parsed_replay)

Load Cell Information


In [84]:
frames[:2]

In [79]:
#all_data = []
_data = parse_replay(parsed_replay, player_names)


Load Basic Information
Load Cell Information
Load Player Ships
Load Other Player Ships
Load Droppoff Information
Load Basic Information
Load Cell Information
Load Player Ships
Load Other Player Ships
Load Droppoff Information
Load Basic Information
Load Cell Information
Load Player Ships
Load Other Player Ships
Load Droppoff Information
Load Basic Information
Load Cell Information
Load Player Ships
Load Other Player Ships
Load Droppoff Information


In [134]:
import datetime

In [150]:
today = datetime.datetime.now() - datetime.timedelta(days=2)
today = str(today).split('-')[:3]
today[-1] = today[-1].split(' ')[0]
today = [int(x) for x in today]

In [151]:
today

[2018, 10, 14]